# 优化问题初步

优化问题的模型由两个部分组成，一个是目标函数，一个是约束条件。建模手的任务就是从赛题抽象出目标函数和约束条件，编程手的任务就是把建模手建立的优化模型求解出来。



## ***第一部分：三种数学规划模型（全是全局最优）***



<br><br>

### **1. 线性规划**

<br><br>


**目标**：最大化$$3x + 2y$$

**约束条件**


- $2x + y \leq 20$
- $4x - 5y \geq -10$
- $x + 2y = 15$
- $0 \leq x \leq 5$
- $0 \leq y \leq 5$


In [6]:
# pulp：一个专门用来实现线性规划、整数规划模型求解的库
import pulp

# 定义问题类型为最大化问题
lp_problem = pulp.LpProblem(name="lp_problem", 
                            sense=pulp.LpMaximize,  # 若模型要求求最小值，则使用pulp.LpMinimize
                            )  

# 定义变量，并设置取值范围
x = pulp.LpVariable("x", lowBound=0, upBound=5)  # 定义变量x，取值范围为[0,5]
y = pulp.LpVariable("y", lowBound=0, upBound=5)  # 定义变量y，取值范围为[0,5]
# z = 
# w = 

# 定义目标函数
lp_problem += 3 * x + 2 * y, "Objective Function"

# 定义不等约束条件
lp_problem += 2 * x + y <= 20, "Constraint 1"
lp_problem += 4 * x - 5 * y >= -10, "Constraint 2"

# 定义等式约束条件
lp_problem += x + 2*y == 15, "Constraint 3"

# 求解线性规划模型，调用pulp库中的求解函数
lp_problem.solve()  

# 输出最优解
print("The optimal value is: ", pulp.value(lp_problem.objective))
print("The optimal solution is:")
for variable in lp_problem.variables():
    print(variable.name, "=", variable.varValue)

The optimal value is:  25.0
The optimal solution is:
x = 5.0
y = 5.0


<br><br>

### **2. 整数规划**

<br><br>






**目标：最大化**  

$$z = 2x + 3y$$

**约束条件：**  

\begin{cases}
x + 2y \leq 8 \\
2x + y \leq 7 \\
x - y = 1 \\
x \geq 0 \\
y \geq 0 \\
x, y \text{ 为整数}
\end{cases}




In [7]:
import pulp

# 创建一个问题实例
prob = pulp.LpProblem("Maximize_2x_3y", pulp.LpMaximize)

# 定义变量，并设置为整数
x = pulp.LpVariable('x', lowBound=0, cat='Integer')
y = pulp.LpVariable('y', lowBound=0, cat='Integer')

# 定义目标函数
prob += 2 * x + 3 * y, "Objective Function"

# 定义约束条件
prob += x + 2*y <= 8, "Constraint 1"
prob += 2*x + y <= 7, "Constraint 2"
prob += x - y == 1, "Constraint 3"

# 求解问题
prob.solve()

# 获取结果
x_value = pulp.value(x)
y_value = pulp.value(y)
objective_value = pulp.value(prob.objective)

print(f"x = {x_value}, y = {y_value}, 最大值 = {objective_value}")

x = 2.0, y = 1.0, 最大值 = 7.0


<br><br>

### **3. 混合型线性规划**

<br><br>





**问题描述：**

**目标：最大化**  
$$z = 3x + 2y + 4z $$

**约束条件：**  

\begin{cases}
2x + y + 3z \leq 10 \\
x + 2y + z \leq 8 \\
x - y + z = 3 \\
x \geq 0 \\
y \geq 0 \\
z \geq 0 \\
x, y \text{ 为整数}, z \text{ 可为实数}
\end{cases}



In [8]:
import pulp

# 创建一个问题实例
prob = pulp.LpProblem("Maximize_3x_2y_4z", pulp.LpMaximize)

# 定义变量，其中 x 和 y 是整数，z 是连续变量
x = pulp.LpVariable('x', lowBound=0, cat='Integer')
y = pulp.LpVariable('y', lowBound=0, cat='Integer')
z = pulp.LpVariable('z', lowBound=0, cat='Continuous')

# 定义目标函数
prob += 3*x + 2*y + 4*z

# 定义约束条件
prob += 2*x + y + 3*z <= 10
prob += x + 2*y + z <= 8
prob += x - y + z == 3

# 求解问题
prob.solve()

# 获取结果
x_value = pulp.value(x)
y_value = pulp.value(y)
z_value = pulp.value(z)
objective_value = pulp.value(prob.objective)

print(f"x = {x_value}, y = {y_value}, z = {z_value}, 最大值 = {objective_value}")

x = 3.0, y = 1.0, z = 1.0, 最大值 = 15.0


<br><br><br>

## ***第二部分：非线性规划模型（极大可能是局部最优）***

当优化模型是凸优化的时候，可以使用梯度下降法求解全局最优。其余情况都是局部最优。

<br><br><br>



**目标**：最小化 $f(x_0, x_1) = 2x_0^2 + 5x_1^2$

**约束条件**：
- $x_0 - 2x_1 + 1 \geq 0$  （不等式约束）
- $x_0^2 + x_1 - 1 \geq 0$  （不等式约束）
- $x_0^2 + x_1 = 1$  （等式约束）
- $x_0 + x_1 = 1$  （等式约束）


In [9]:
from scipy.optimize import minimize

# 不等式约束：默认大于等于
ineq_cons = [
                {
                'type': 'ineq', 
                'fun': lambda x: x[0] - 2 * x[1] + 1
                },

                {
                'type': 'ineq', 
                'fun': lambda x: x[0] ** 2 + x[1] - 1
                }
        ]

# 等式约束：默认等于0
eq_cons = [
                {
                'type': 'eq', 
                'fun': lambda x: x[0] ** 2 + x[1] - 1
                },

                {
                'type': 'eq', 
                'fun': lambda x: x[0] + x[1] - 1
                }
        ]

# 约束组合
constraints = ineq_cons + eq_cons

# 进行求解
res = minimize(
        fun=lambda x: 2 * x[0] ** 2 + 5 * x[1] ** 2, 
        x0=[0, 0],  # 初始点的选择
        constraints=constraints,
        method='SLSQP'
        )

# 输出最优解
print("The optimal value is: ", res.fun)
print("The optimal solution is:", res.x)

The optimal value is:  5.0
The optimal solution is: [-2.22044575e-17  1.00000000e+00]


<br><br><br>

## ***第三部分：多目标规划模型***

<br><br><br>

**目标**：最小化 $f_1(x_0, x_1) = (x_0-2)^2$的同时最小化 $f_2(x_0, x_1) = (x_1+3)^2$

**约束条件**：
- $x_0 - 2x_1 + 1 \geq 0$  （不等式约束）
- $x_0^2 + x_1 - 1 \geq 0$  （不等式约束）
- $x_0^2 + x_1 = 1$  （等式约束）
- $x_0 + x_1 = 1$  （等式约束）


In [10]:
from scipy.optimize import minimize

# 函数1
def f1(x):
    return (x[0] - 2) ** 2

# 函数2
def f2(x):
    return (x[1] + 3) ** 2

# 综合目标函数
def objective_function(x, w1=0.5, w2=0.5):
    return w1 * f1(x) + w2 * f2(x)

# 不等式约束
ineq_cons = [{'type': 'ineq', 'fun': lambda x: x[0] - 2*x[1] + 1},
              {'type': 'ineq', 'fun': lambda x: x[0]**2 + x[1] - 1},
              ]

# 等式约束
eq_cons = [{'type': 'eq', 'fun': lambda x: x[0]**2 + x[1] - 1},
             {'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1},
              ]

# 组合约束
constraints = ineq_cons + eq_cons



res = minimize(
            fun=objective_function, 
            x0=[0, 0], 
            args=(0.5, 0.5),
            constraints=constraints,
            method='SLSQP'
            )

print('最优解：', res.x)
print('最优值：', res.fun)

最优解： [3.3087225e-24 1.0000000e+00]
最优值： 10.0
